In [ ]:
-from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 기본 라이브러리
import seaborn as sns
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
pd.set_option('display.max_colwidth', -1) # 데이터프레임 출력시 생략없이 다 나오게 하는 기능
from google.colab.patches import cv2_imshow
import cv2
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


# LightGBM

In [ ]:
# LightGBM의 파이썬 패키지인 lightgbm에서 LGBMClassifier 임포트
from lightgbm import LGBMClassifier

import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

#연속형 변수 스케일링
scaler = StandardScaler()

In [ ]:
path = "/content/drive/Shareddrives/기업연계 NEW_6/data/트리계열필요데이터"
file_lst = os.listdir(path)

In [ ]:
#setting up the parameters
params={}
params['learning_rate']=0.02
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multiclass' #metric for multi-class
params['max_depth']=50
params['num_class']=5 #no.of unique values in the target class not inclusive of the end value

In [ ]:
# result of model
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix, accuracy_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = multilabel_confusion_matrix( y_test, pred)
    num_class = confusion.shape[0]
    l = []
    a = []
    total_f1 = []
    #print(confusion)
    for i in range(num_class):
        
        print("class",i,"의 결과")
        
        TP = confusion[i][0][0] 
        FP =  confusion[i][0][1] 
        FN =  confusion[i][1][0] 
        TN =  confusion[i][1][1] 
        accuracy = (TP+TN)/(TP+TN+FP+FN)
        precision = TP/(TP+FN)
        recall = TP/(TP+FP)
        f1 = 2*(precision*recall)/(precision+recall)
        total_f1.append(f1)
        a.append(TP+FP+FN+TN)
        l.append(TP)
        l.append(TN)
        print('accuracy :', accuracy, 'precision :', precision,'recall :', recall,'f1 :', f1)
        
    total_accuracy = sum(l)/sum(a)
    print("전체 accuracy :", total_accuracy)
    return total_f1

In [ ]:
# feature importacne 확인
from lightgbm import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

## 정면

In [ ]:
# 정면
df = pd.read_csv("/content/drive/Shareddrives/기업연계 NEW_6/data/트리계열필요데이터/front_all_label.csv",index_col=0)

# X_features
X_features = df[[  'euclidean', 'cosine', '0',  '1','2',  '3','4',  '5','6', '7', '8', '9','10',  '11', '12', '13',
'14',  '15', '16','17'] ].to_numpy(dtype=None, copy=False)

# y_label별 분포 확인
from collections import Counter
print(Counter(df['label'])) 
print(Counter(df['mode_trim_round'])) 
print((Counter(df['only_trim_round'])))

label_f1_frontface = {'label': [], 'mode_trim_round':[], 'only_trim_round':[]}
# y_label 세가지 중 택 1
for label in ['label', 'mode_trim_round', 'only_trim_round']:
    y_label = df[ label].to_numpy(dtype=None, copy=False) 
    # 스케일링
    sc = StandardScaler()
    sc.fit(X_features)
    X_features_sc =pd.DataFrame(sc.fit_transform(X_features))
    # 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
    X_train, X_test, y_train, y_test=train_test_split(X_features_sc, y_label,
                                            test_size=0.2, random_state=2022, stratify= y_label)
    print(X_train.shape , X_test.shape)
    # 앞서 XGBoost와 동일하게 n_estimators는 400 설정. 
    lgbm_wrapper = LGBMClassifier(learning_rate=0.02,boosting_type='gbdt',objective='multiclass',metric='multiclass',max_depth=50,num_class=5) #n_estimators=400, 

    # LightGBM도 XGBoost와 동일하게 조기 중단 수행 가능. 
    evals = [(X_test, y_test)]
    lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, 
                    eval_set=evals, verbose=True)

    preds = lgbm_wrapper.predict(X_test)
    pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]
    print("레이블을 ", label, "으로 선택시 성능")
    print(get_clf_eval(y_test, preds) )
    label_f1_frontface[label] = get_clf_eval(y_test, preds)
    fig, ax = plt.subplots(figsize=(10, 12))
    plot_importance(lgbm_wrapper, ax=ax)
    import joblib
    filename = '/content/drive/Shareddrives/기업연계 NEW_6/lightGBM_model_saved/lightGBM_frontface_model_{0}.pkl'.format(label)              
    joblib.dump(lgbm_wrapper, filename) 
    


## 측면

In [ ]:
# 측면
df_r = pd.read_csv("/content/drive/Shareddrives/기업연계 NEW_6/data/트리계열필요데이터/right_all_label.csv",index_col=0)
df_l = pd.read_csv("/content/drive/Shareddrives/기업연계 NEW_6/data/트리계열필요데이터/left_all_label.csv",index_col=0)
df = pd.concat([df_r, df_l])

# X_features 
X_features = df[[  'euclidean', 'cosine', '0',  '1','2',  '3','4',  '5','6', '7', '8', '9','10',  '11', '12', '13',
'14',  '15', '16'] ].to_numpy(dtype=None, copy=False)

# y_label별 분포 확인
from collections import Counter
print(Counter(df['label'])) 
print(Counter(df['mode_trim_round'])) 
print((Counter(df['only_trim_round']))) 


label_f1_sideface = {'label': [], 'mode_trim_round':[], 'only_trim_round':[]}
# y_label 세가지 중 택 1
for label in ['label', 'mode_trim_round', 'only_trim_round']:
    y_label = df[ label].to_numpy(dtype=None, copy=False) 
    # 스케일링
    sc1 = StandardScaler()
    sc1.fit(X_features)
    X_features_sc =pd.DataFrame(sc.fit_transform(X_features))
    # # 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
    X_train, X_test, y_train, y_test=train_test_split(X_features_sc, y_label,
                                            test_size=0.2, random_state=2022, stratify= y_label)
    print(X_train.shape , X_test.shape)
    
    lgbm_wrapper = LGBMClassifier(n_estimators=400)
    
    
    evals = [(X_test, y_test)]
    lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, 
                    eval_set=evals, verbose=True)

    preds = lgbm_wrapper.predict(X_test)
    pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]
    print("레이블을 ", label, "으로 선택시 성능")
    print(get_clf_eval(y_test, preds) )
    label_f1_sideface[label] = get_clf_eval(y_test, preds)
    fig, ax = plt.subplots(figsize=(10, 12))
    plot_importance(lgbm_wrapper, ax=ax)
    import joblib
    filename = '/content/drive/Shareddrives/기업연계 NEW_6/lightGBM_model_saved/lightGBM_model_sideface_{0}.pkl'.format(label)              
    joblib.dump(lgbm_wrapper, filename)

# y label 결정

          #f1 of front 
          'label': [0.43478260869565216, 0.7524752475247526, 0.9487516425755586, 0.8815977175463624, 0.9834815756035579], 
          'mode_trim_round': [0.8450704225352113, 0.5174129353233831, 0.9005524861878452, 0.84080370942813, 0.9847715736040609], 
          'only_trim_round': [0.9425901201602136, 0.3795620437956204, 0.8579795021961933, 0.8981348637015781, 0.9962358845671266] 
          #f1 of side
           'label': [nan, 0.8830715532286213, 0.93687707641196, 0.     9386401326699835, 0.9937106918238994], 
           'mode_trim_round': [0.832391713747646, 0.12440191387559808, 0.9022298456260721, 0.935108153078203, 0.9937106918238994], 
           'only_trim_round': [0.9168081494057725, 0.10404624277456648, 0.8397790055248618, 0.9610389610389611, 0.9984350547730829]

 결과를 통해 mode_trim_round로 결정 




In [ ]:
from sklearn.model_selection import RandomizedSearchCV

y_label = df['mode_trim_round'].to_numpy(dtype=None, copy=False) 
# 스케일링
sc = StandardScaler()
sc.fit(X_features)
X_features_sc =pd.DataFrame(sc.fit_transform(X_features))
# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test=train_test_split(X_features_sc, y_label,
                                        test_size=0.2, random_state=2022, stratify= y_label)

rf = LGBMClassifier()
rf.fit(X_train, y_train)

hyperparamters = {'max_depth': np.arange(1, 80,10),   
              'min_samples_split': np.arange(1, 80, 10), 
              'min_samples_leaf': np.arange(1, 100, 10), 
              'max_leaf_nodes': np.arange(1, 100, 10)}

# iteration 횟수 설정
n_iter = 1000

# RandomizedSearch 진행
rs = RandomizedSearchCV(rf,
                    param_distributions=hyperparamters, 
                    n_iter = n_iter,
                    cv=5, verbose=3,
                    return_train_score = True)

rs.fit(X_train, y_train)


In [ ]:
rs.best_params_ # 가장 좋은 성능을 보였을 때의 parameter

{'max_depth': 1,
 'max_leaf_nodes': 11,
 'min_samples_leaf': 31,
 'min_samples_split': 71}

# best params에 의한 모델

In [ ]:
# 정면
df = pd.read_csv("/content/drive/Shareddrives/기업연계 NEW_6/트리계열필요데이터/front_all_label.csv",index_col=0)

# X_features
X_features = df[[  'euclidean', 'cosine', '0',  '1','2',  '3','4',  '5','6', '7', '8', '9','10',  '11', '12', '13',
'14',  '15', '16','17'] ].to_numpy(dtype=None, copy=False)

# y_label별 분포 확인
from collections import Counter
print(Counter(df['label'])) 
print(Counter(df['mode_trim_round'])) 
print((Counter(df['only_trim_round'])))

label_f1_frontface = {'label': [], 'mode_trim_round':[], 'only_trim_round':[]}
# y_label 세가지 중 택 1
for label in ['mode_trim_round']:
    y_label = df[ label].to_numpy(dtype=None, copy=False) 
    # 스케일링
    sc = StandardScaler()
    sc.fit(X_features)
    X_features_sc =pd.DataFrame(sc.fit_transform(X_features))
    # 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
    X_train, X_test, y_train, y_test=train_test_split(X_features_sc, y_label,
                                            test_size=0.2, random_state=2022, stratify= y_label)
    print(X_train.shape , X_test.shape)
    # 앞서 XGBoost와 동일하게 n_estimators는 400 설정. 
    lgbm_wrapper = LGBMClassifier(learning_rate=0.02,boosting_type='gbdt',objective='multiclass',metric='multiclass',max_depth =  1,
 max_leaf_nodes= 11,
 min_samples_leaf= 31,
 min_samples_split= 71) #n_estimators=400, 

    # LightGBM도 XGBoost와 동일하게 조기 중단 수행 가능. 
    evals = [(X_test, y_test)]
    lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, 
                    eval_set=evals, verbose=True)

    preds = lgbm_wrapper.predict(X_test)
    pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]
    print("레이블을 ", label, "으로 선택시 성능")
    print(get_clf_eval(y_test, preds) )
    label_f1_frontface[label] = get_clf_eval(y_test, preds)
    fig, ax = plt.subplots(figsize=(10, 12))
    plot_importance(lgbm_wrapper, ax=ax)
    import joblib
    filename = '/content/drive/Shareddrives/기업연계 NEW_6/lightGBM_model_saved/lightGBM_frontface_model_{0}.pkl'.format(label)              
    joblib.dump(lgbm_wrapper, filename) 
    